# Tutorial for PsPolypy

## Imports

In [ ]:
# Import necessary libraries for this example
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Locate PsPolypy. Only necessary if PsPolypy is not installed as a package.
import sys
sys.path.append('../src/')

# Import PsPolypy
import  PsPolypy

## The Polydat Object

### Overview

```Polydat``` is object that stores and processes image fields containing polymer particles. The object can handle any number of full field images, provided they are the same nm/px resolution. A general outline of the ```Polydat```'s attributes are shown here:

```project
Polydat
|- images (list[numpy.ndarray]):
|     A list containing numpy arrays for each loaded full field image.
|- resolution (float):
|     The images' resolution in nm/px.
|- metadata (Dict[str, any]):
|     Dictionary containing key value pairs of arbitrary metadata. For example, the date of data acquisition could be stored as {'recorded_on':'12/25/24'}.
|- particles (list[Particle]):
|     List of Particle objects. Each Particle represents a detected polymer particle from the full field image. Set by the segment_particles method.
|- num_particles (int):
|     The total number of polymer particles detected in the full field images. Set by the segment_particles method.
|- contour_lengths (list[float]):
|     List containing the contour lengths for all branches detected in the particle list. Set by the interpolate_particles method.
|- contour_sampling (numpy.ndarray):
|     1-D array of interpolated contour sampling points. Ranges from 0 to the maximum detected contour length in a user defined step size. Set by the interpolate_particles method.
|- mean_tantan_correlation (np.ndarray):
|     1-D array of the tangent-tangent correlation averaged over all particle branches. Set by the calc_tantan_correlations method.
|- pl (float):
|     The persistence length of the polymer. Set by the calc_persistence_length method.
|- plcov (float):
|     The covariance of the persistence length fit. Set by the calc_persistence_length method.
```

### Creating an instance of Polydat

There are three means of creating an instance of the ```Polydat``` class. The first is to use the built-in class method ```from_images``` for creating an instance of ```Polydat``` from a list of image files. Additionally, you may wish to create an empty instance of the ```Polydat``` class and add images with the ```add_image``` method. The final option is to load the image to an array and pass it in to the class init. The three options are shown below.

In [ ]:
# Create an instance of the Polydat class from a single image. Note, it must be a list of file paths, even if there is only one image.
image_path = ['example_images/exampleCL0.png']
polydat = PsPolypy.Polymer.Polydat.from_images(image_path, resolution = 2)
# Check that the polydat object has been created correctly.
print(f'The first polydat object contains {len(polydat.images)} image(s).')

# Example of creating an instance of the Polydat class from a list of image file paths.
image_paths = ['example_images/exampleCL0.png', 'example_images/exampleCL1.png']
polydat = PsPolypy.Polymer.Polydat.from_images(image_paths, resolution = 2)
# Check that the polydat object has been created correctly.
print(f'The second polydat object contains {len(polydat.images)} image(s).')

In [ ]:
# Create an empty instance of the Polydat class and add an image to it.
polydat = PsPolypy.Polymer.Polydat()
polydat.add_image('example_images/exampleCL0.png', resolution = 2)
polydat.add_image('example_images/exampleCL1.png', resolution = 2)
# Check that the polydat object has been created correctly.
print(f'The polydat object contains {len(polydat.images)} image(s).')

In [ ]:
# Create an instance of the Polydat class from a list of numpy arrays.
filepath = 'example_images/exampleCL0.png'
with Image.open(filepath) as img:
    grayscale = img.convert('L')
    image_data = np.array(grayscale) / 255.0
polydat = PsPolypy.Polymer.Polydat([image_data], resolution = 2)
# Check that the polydat object has been created correctly.
print(f'The polydat object contains {len(polydat.images)} image(s).')

### Processing images with Polydat

With a loaded instance of the ```Polydat``` class, the full field polymer image is ready to be processed.

The first method for processing the image(s) is to (optionally) upscale them. The ```upscale``` method takes two inputs, ```magnification``` (int/float) and ```order``` (int). Each image has its shape multiplied by the ```magnification``` factor and interpolated by a 2D spline of ```order``` power. The upscaled image is set set back in the ```images``` attribute, i.e. this operation occurs in place. Furthermore, the ```resolution``` attribute is dynamically updated to reflect the new nm/px value. 

Note:

The ```upscale``` method will raise an exception if the ```upscaled``` metadata is ```True```. This prevents upscaled images from being upscaled multiple times. After ```upscale``` is executing the ```metadata``` will be automatically updated with the key-value pair ```upscaled: True```. 

Though ```magnification```may be a floating point number, this may lead to warped images and unexpected behavior; it is preferrable to keep the magnification factor an integer. The interpolation ```order``` must be an integer from 0 to 5, indicating nearest-neighbor to bi-quintic interpolation. The default interpolation order is 3: bi-cubic. 

In [ ]:
# Create an instance of the Polydat class from the test image.
image_path = ['example_images/exampleCL0.png']
polydat = PsPolypy.Polymer.Polydat.from_images(image_path, resolution = 2)
# Extract the image from the polydat object.
base_image = polydat.images[0]

# Upscale the image by a factor of 2 using bi-cubic interpolation.
polydat.upscale(magnification = 2, order = 3)
# Extract the upscaled image from the polydat object.
upscaled_image = polydat.images[0]

# Plot the original and upscaled images for comparison.
fig, ax = plt.subplots(1,2, figsize = (7,6))
for axis in ax:
    axis.axis('off')

# Display the images.
ax[0].imshow(base_image, cmap = 'gray')
ax[0].set_title(f'Original {base_image.shape}')
ax[1].imshow(upscaled_image, cmap = 'gray')
ax[1].set_title(f'Upscaled {upscaled_image.shape}')

# Display the plot.
plt.show()

Whether or not the ```images``` are interpolated, we can proceed to detecting and segmenting the particles. This is done by executing the ```segment_particles``` method of the ```Polydat``` instance. This method takes two optional arguments: ```minimum_area``` (int) and ```padding``` (int). During the segmentation process, any particle whose area is less than ```minimum_area``` is discarded. When the particle images are cropped, the ```padding``` argument indicates how many padding pixels to include around the cropped region.

In [ ]:
# Segment the particles in the image.
polydat.segment_particles()

# Check to see how many particles were detected.
print(f'The polydat object detected and segmented {polydat.num_particles} particles.')


After segmentation, the ```particles``` attribute is set. It is a list containing ```Particle``` objects. A general outline of the relevant ```Particle``` attributes is shown here.

```project
Particle
|- image (numpy.ndarray):
|     The image of the particle.
|- resolution (float):
|     The images' resolution in nm/px. Inherited from Polydat class.
|- bbox (Tuple[int,int,int,int]):
|     The coordinates for the bounding box surrounding the particle.
|- binary_mask (numpy.ndarray):
|     The binary mask selecting the particle.
|- classification (str):
|     The particle classification. Linear, Branched, Branched-Loop, Looped, or Unknown
|- particle_id (int):
|     Unique identification number for the particle. Assigned by skimage.measure.regionprops.
|- skeleton (skan.Skeleton):
|     skan.Skeleton object of the particle.
|- skelton_summary (pandas.DataFrame):
|     The DataFrame created from skan.Summarize()
|- contour_samplings (list[numpy.ndarray]):
|     A list of the contour sampling for each path in the particle.
|- contour_lengths (list[float]):
|     A list of the contour length for each path in the particle.
|- interp_skeleton_coordinates (list[numpy.ndarray]):
|     The coordinates of each path of the particle's skeleton interpolated according to the contour sampling.
|- interp_skeleton_derivatives (list[numpy.ndarray]):
|     The derivatives of each interpolated path of the particle's skeleton.
|- tantan_correlations (list[numpy.ndarray]):
|     The tangent-tangent correlation of each interpolated path of the particle's skeleton.
```

With the particles now segmented, let's visualize one using the ```plot_particle``` method. It takes the particle ```index```. Let's plot particle 0.

**Note:** All plotting functions take the optional argument ```ax``` which chooses the ```matplotlib.pyplot.axis``` to draw to. If not explicitly specified, the current axis is selected with ```matplotlib.pyplot.gca```. For more information, see the Matplotlib documentation [here](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.gca.html).

In [ ]:
# Select which particle to view the image of.
particle_index = 10

# Create the figure and the axis for the particle image.
particle_fig, ax = plt.subplots(figsize = (6,6))
# Plot the image to the axis.
ax = polydat.plot_particle(particle_index, cmap = 'gray')
# Set the title of the plot.
ax.set_title(f'Particle {particle_index} Image')
# Turn the axis off
ax.axis('off')
# Display the plot.
plt.show()

In [ ]:
# Skeletonize the particles with the default parameters.
polydat.skeletonize_particles()

With the skeleton now set, we can view the particle skeletons with the ```plot_skeleton``` method. It functions similarly to the previous plotting method, taking in the ```index``` of the particle whose sekeleton to plot.

In [ ]:
# Select which particle to view the skeleton of.
particle_index = 10

# Create the figure and the axis for the skeleton image.
skeleton_fig1, ax = plt.subplots(figsize = (6,6))
# Plot the skeleton image to the axis.
ax = polydat.plot_skeleton(particle_index, cmap = 'gray')
# Set the title of the plot.
ax.set_title(f'Particle {particle_index} Skeleton')
# Turn the axis off
ax.axis('off')
# Display the plot.
plt.show()

The particle skeletons can now be interpolated with a 2D interpolating spline. The ```interpolate_skeletons``` method interpolates each particle skeleton along the contur in a user defined ```step_size``` (float). ```step_size``` should be set below 1 to sample the contour above the current pixel resolution. The optional arguments for the interpolation order ```k``` (int) and smoothing ```s``` (float) are passed to ```scipy.interpolate.splprep```. For more information see the scipy documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.splprep.html#scipy.interpolate.splprep).

This sets the ```contour_lengths```, ```contour_samplings```, ```interp_skeleton_coordinates```, and ```interp_skeleton_derivatives``` attributes. Furthermore, the ```plot_interpolated_skeleton``` method will now display the interpolated skeleton contour.

In [ ]:
# Interpolate the skeletons of the particles.
polydat.interpolate_skeletons(step_size = 0.5, k = 3, s = 0.5)

# Select which particle to view the interpolated skeleton of.
particle_index = 10

# Create the figure and the axis for the skeleton image.
skeleton_fig2, ax = plt.subplots(figsize = (6,6))
# Plot the skeleton image to the axis.
ax = polydat.plot_skeleton(particle_index, cmap = 'gray')
# Plot the interpolated skeleton to the axis.
ax = polydat.plot_interpolated_skeleton(particle_index, lw = 6, color = 'red')
# Set the title of the plot.
ax.set_title(f'Particle {particle_index} Interpolated Skeleton')
# Turn the axis off
ax.axis('off')
# Display the plot.
plt.show()

With the ```contour_lengths``` attribute set, we can now plot a distribution of the contour lengths of all particle paths. This is done by executing the ```plot_contour_distribution``` method. This plotting takes in ```Dict```s of keyword arguments passed to matplotlib. Because we're plotting the full contour length distribution, the kwargs for controling the plot are ```inc_dist_kwargs``` and ```inc_fill_kwargs``` for the distribtuion and fill respectively. See the docstring for more information.

In [ ]:
# Create the figure and the axis for the contour distribution.
contour_distribution_fig1, ax = plt.subplots(figsize = (6,6))

# Plot the contour length distribution.
ax = polydat.plot_contour_distribution(n_points = 1000,
                                       inc_dist_kwargs = {'color': 'Blue', 'lw': 2,},
                                       inc_fill_kwargs = {'color': 'LightBlue', 'alpha': 0.5})
# Set the title of the plot.
ax.set_title('Contour Length Distribution')
# Set the axis labels.
ax.set_xlabel(f'Contour Length (x{polydat.resolution} nm)')
ax.set_ylabel('Probability Density')
# Set the axis limits.
ax.set_ylim([0,0.085])
ax.set_xlim([0,75])
plt.show()

### Classifying The Particles

Once the```skeleton```s are set, the particles can be classified by executing the ```classify_particles``` method. This sets the ```classification``` attribute. Each particle is classified by the following criteria:
- If the particle contains a single branch path with two different endpoints, the particle is classified as ```Linear```.
- If the particle contains a single branch path with the same end points, the particle is classified as ```Looped```.
- If the particle contains multiple branch paths that intersect at a branch point but no cycles (A point can be revisited when traveling along a set of branch paths), the particle is classified as ```Branched```.
- If the particle contains multiple branch paths that intersect at a branch point and includes cycles, the particle is classified as ```Brached-Looped```.
- If none of the above criteria are met, the particle is classified as ```Unknown```. This should not occur, but is included for safety.

Once the skeletons are classified, you can return a subset of the ```particles``` list attribute using the ```get_filtered_particles``` method passing in the ```filter_str``` (str) matching the particles type you want. Additionally, the ```plot_particle``` and ```plot_skeleton``` methods will automatically display the classification.

In [ ]:
# Classify the Particles
polydat.classify_particles()

# Get the number of particles in the Linear classification.
linear_particles = polydat.get_filtered_particles('Linear')
print(f'The number of linear particles is {len(linear_particles)}.')

# Select the first particle in the Linear classification.
particle = linear_particles[0]

# Create the figure and the axis for the particle image.
linear_skeleton_fig, ax = plt.subplots(figsize = (6,6))
# Plot the first particle's skeleton.
ax = particle.plot_skeleton(cmap = 'gray')
# Plot the first particle's interpolated skeleton.
ax = particle.plot_interpolated_skeleton(lw = 6, color = 'red')
# Set the title of the plot.
ax.set_title(f'Particle {particle.id} Interpolated Skeleton - {particle.classification}')
# Turn the axis off
ax.axis('off')

### Calculating the Persistence Length

Once the interpolated skeleton is set, the mean tangent-tangent correlation is calculated by executing the ```calc_tantan_correlations``` method. The tangent-tangent correlation is calculated for each branch in all particles and stored in the particles' ```tantan_correlations``` attribute. The user can also calculate the tangent-tangent correlations for a set of specific particle classifications by passing in the ```included_classifications``` argument. The correlations are abs-averaged for each contour length in ```contour_samplings```, the aboslute value is taken, and the output is set to the ```mean_tantan_correlation``` attribute. 

Once the ```mean_tantan_correlation``` is set, we can plot it with the ```plot_mean_tantan_correlation``` method. It takes a single optional argument ```error_bars``` (bool) to turn error bars on and off. Furthermore, the plotting function includes keyword argument dictionaries ```inc_kwargs``` and ```exc_kwargs``` for altering the color of the correlation, its errorbars, etc. See the source code for more information.

In [ ]:
# Calculate the tangent-tangent correlation functions for all the particles.
polydat.calc_tantan_correlations(included_classifications = ['Linear', 'Branched', 'Branched-Looped', 'Looped'])

tantan_correlation_fig1, ax = plt.subplots(figsize = (6,6))
# Plot the mean tangent-tangent correlation function.
ax = polydat.plot_mean_tantan_correlation(error_bars = True,
                                          inc_kwargs = {'color': 'Blue', 'fmt': '.', 'ecolor': 'LightBlue'},
                                          exc_kwargs = {'color': 'Gray', 'fmt': '.', 'ecolor': 'LightGray'})
# Set the title of the plot.
ax.set_title('Mean Tangent-Tangent Correlation')
# Set the axis labels.
ax.set_xlabel(f'Distance (x{polydat.resolution} nm)')
ax.set_ylabel('abs <Correlation>')
# Set the axis limits.
ax.set_xlim(0,60)
ax.set_ylim(0,1.05)
# Turn the grid on.
ax.grid()
# Display the plot.
plt.show()

Finally, calculating the persistence length can be done with the ```calc_tantan_lp``` method. It accepts two optional arguments, ```min_fitting_length``` (float) and ```max_fitting_length``` (float). Because the correlation contribution from low contour lengths can bias the fit, and the sampling of very long contour lengths is very low, the decaying exponential is only fit between these two values. Executing ```calc_tantan_lp``` sets the ```tantan_fit_result``` parmeter with a ```lmfit.model.ModelResult```. For more information about the result see the documentation [here](https://lmfit.github.io/lmfit-py/model.html#lmfit.model.ModelResult).

Once the persistence length is calculated, we can see a summary of the polydat object with ```print_summary``` method. The ```plot_mean_tantan_correlation_fit``` method plots the fitted exponential. If the ```show_init``` argument is ```True```, the initial guess at the fit is shown along side the best fit. The ```fit_kwargs``` and ```init_kwargs``` can be set to adjust the parameters of the fit and initial guess plots.

In [ ]:
# Calculate the persistence length of the particles.
polydat.calc_tantan_lp(min_fitting_length = 7, max_fitting_length = 40)

# Print a summary of the polydat object.
polydat.print_summary()

tantan_correlation_fig2, ax = plt.subplots(figsize = (6,6))
# Plot the mean tangent-tangent correlation function.
ax = polydat.plot_mean_tantan_correlation(error_bars = True,
                                          inc_kwargs = {'color': 'Blue', 'fmt': '.', 'ecolor': 'LightBlue', 'label': 'Fitted Data'},
                                          exc_kwargs = {'color': 'Gray', 'fmt': '.', 'ecolor': 'LightGray', 'label': 'Excluded Data'})
# Plot the fitted decaying exponential.
ax = polydat.plot_mean_tantan_correlation_fit(show_init = True,
                                              fit_kwargs = {'color': 'Red', 'lw': 1.5, 'label': 'Best Fit'},
                                              init_kwargs = {'color': 'Red', 'lw': 0.75, 'linestyle': '--', 'label': 'Initial Guess'})
# Set the title of the plot.
ax.set_title('Mean Tangent-Tangent Correlation')
# Set the axis labels.
ax.set_xlabel(f'Distance (x{polydat.resolution} nm)')
ax.set_ylabel('abs <Correlation>')
ax.legend()
# Set the axis limits.
ax.set_xlim(0,60)
ax.set_ylim(0,1.05)
# Turn the grid on.
ax.grid()
# Display the plot.
plt.show()

In [ ]:
polydat.tantan_fit_result

## Complete Workflow

Below is an example of the end-to-end workflow for processing a set of polymer images for all classification types.

In [ ]:
%%time
# Create a list of the files to be analyzed.
filepaths = ['example_images/exampleCL0.png', 'example_images/exampleCL1.png']
# Create an instance of the Polydat class from the list of file paths.
polydat = PsPolypy.Polymer.Polydat.from_images(filepaths = filepaths, resolution = 2)
# Upscale the image by a factor of 2 using bi-cubic interpolation.
polydat.upscale(magnification = 2, order = 3)
# Segment the particles in the image.
polydat.segment_particles()
# Skeletonize the particles.
polydat.skeletonize_particles()
# Classify the particles.
polydat.classify_particles()
# Interpolate the skeletons of the particles.
polydat.interpolate_skeletons(step_size = 0.5, k = 3, s = .5)
# Calculate the tangent-tangent correlation for the particles.
polydat.calc_tantan_correlations()
# Calculate the persistence length.
polydat.calc_tantan_lp(min_fitting_length = 7, max_fitting_length = 40)
# Print a summary of the polydat object.
polydat.print_summary()

# Create a figure containing the complete workflow plots.
complete_workflow_fig, ax = plt.subplots(2,1, figsize = (7,11))

# Plot contour length distribution
ax[0] = polydat.plot_contour_distribution(n_points = 1000, ax = ax[0],
                                          inc_dist_kwargs = {'color': 'Blue', 'lw': 2, 'label': 'Included Distribution'},
                                          inc_fill_kwargs = {'color': 'LightBlue', 'alpha': 0.5},
                                          exc_dist_kwargs = {'color': 'Gray', 'lw': 2, 'alpha': 0.5, 'label': 'Excluded Distribution'},
                                          exc_fill_kwargs = {'color': 'LightGray', 'alpha': 0.5},
                                          vline_kwargs = {'color': 'Blue', 'lw': 1.5, 'linestyle': '--'})
ax[0].set_xlim([0,60])
ax[0].set_ylim([0,0.065])
ax[0].grid(lw=0.3)
ax[0].legend()

# Plot tangent-tangent correlation with the fit.
ax[1] = polydat.plot_mean_tantan_correlation(error_bars = True, ax = ax[1],
                                             inc_kwargs = {'color': 'Blue', 'fmt': '.', 'ecolor': 'LightBlue', 'label': 'Fitted Data'},
                                             exc_kwargs = {'color': 'Gray', 'fmt': '.', 'ecolor': 'LightGray', 'label': 'Excluded Data'})
ax[1] = polydat.plot_mean_tantan_correlation_fit(show_init = True, ax = ax[1],
                                                 fit_kwargs = {'color': 'Red', 'lw': 1.5, 'label': 'Best Fit'},
                                                 init_kwargs = {'color': 'Red', 'lw': 0.75, 'linestyle': '--', 'label': 'Initial Guess'})
ax[1].set_xlim([0,60])
ax[1].set_ylim(0,1.05)
ax[1].grid(lw=0.3)
ax[1].legend()

# Display the plot.
plt.show()
# Show the fitting results.
polydat.tantan_fit_result

Below is an example of the end-to-end workflow for processing a set of polymer images only considering the linear classification.

In [ ]:
# Create a list of the files to be analyzed.
filepaths = ['example_images/exampleCL0.png', 'example_images/exampleCL1.png']
# Create an instance of the Polydat class from the list of file paths.
polydat = PsPolypy.Polymer.Polydat.from_images(filepaths = filepaths, resolution = 2)
# Upscale the image by a factor of 2 using bi-cubic interpolation.
polydat.upscale(magnification = 2, order = 3)
# Segment the particles in the image.
polydat.segment_particles()
# Skeletonize the particles.
polydat.skeletonize_particles()
# Classify the particles.
polydat.classify_particles()
# Interpolate the skeletons of the particles.
polydat.interpolate_skeletons(step_size = 0.5, k = 3, s = .5)
# Calculate the tangent-tangent correlation for the particles. Include only the Linear classification.
polydat.calc_tantan_correlations(included_classifications = ['Linear'])
# Calculate the persistence length.
polydat.calc_tantan_lp(min_fitting_length = 7, max_fitting_length = 40)
# Print a summary of the polydat object.
polydat.print_summary()

# Create a figure containing the complete workflow plots.
complete_workflow_fig, ax = plt.subplots(2,1, figsize = (7,11))

# Plot contour length distribution
ax[0] = polydat.plot_contour_distribution(n_points = 1000, ax = ax[0],
                                          inc_dist_kwargs = {'color': 'Blue', 'lw': 2, 'label': 'Included Distribution'},
                                          inc_fill_kwargs = {'color': 'LightBlue', 'alpha': 0.5},
                                          exc_dist_kwargs = {'color': 'Gray', 'lw': 2, 'alpha': 0.5, 'label': 'Excluded Distribution'},
                                          exc_fill_kwargs = {'color': 'LightGray', 'alpha': 0.5},
                                          vline_kwargs = {'color': 'Blue', 'lw': 1.5, 'linestyle': '--'})
ax[0].set_xlim([0,60])
ax[0].set_ylim([0,0.065])
ax[0].grid(lw=0.3)
ax[0].legend()

# Plot tangent-tangent correlation with the fit.
ax[1] = polydat.plot_mean_tantan_correlation(error_bars = True, ax = ax[1],
                                             inc_kwargs = {'color': 'Blue', 'fmt': '.', 'ecolor': 'LightBlue', 'label': 'Fitted Data'},
                                             exc_kwargs = {'color': 'Gray', 'fmt': '.', 'ecolor': 'LightGray', 'label': 'Excluded Data'})
ax[1] = polydat.plot_mean_tantan_correlation_fit(show_init = True, ax = ax[1],
                                                 fit_kwargs = {'color': 'Red', 'lw': 1.5, 'label': 'Best Fit'},
                                                 init_kwargs = {'color': 'Red', 'lw': 0.75, 'linestyle': '--', 'label': 'Initial Guess'})
ax[1].set_xlim([0,60])
ax[1].set_ylim(0,1.05)
ax[1].grid(lw=0.3)
ax[1].legend()

# Display the plot.
plt.show()
# Show the fitting results
polydat.tantan_fit_result

## Versions of used packages 

In [ ]:
%load_ext watermark
%watermark --python --packages numpy,scipy,networkx,skan,matplotlib,PIL,skimage,lmfit